In [25]:
from docx import Document
import pandas as pd
import re
from collections import defaultdict
import urllib
from os import listdir
from os.path import isfile, join, abspath

In [37]:

datafiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]
columns = ['Дата', 'Адрес', 'Оксид углерода', 'Оксид азота', 'Диоксид азота', 'Диоксид серы', 'Озон', 'Взвешенные частицы PM10', 'Взвешенные частицы PM2.5']

In [53]:
def addDate(fn):
    df = pd.DataFrame(columns=columns)
    with open(abspath('data/'+fn), 'rb') as f:
        pollution_report = Document(f)

        adr_cnt = 0
        for i, paragraph in enumerate(pollution_report.paragraphs):
            row_df = dict.fromkeys(columns)
            adress = re.findall("адресу: (.*?)\\(станции", paragraph.text)
            if adress:
                row_df['Адрес'] = adress[0]
                row_df['Дата']  = fn.split('.')[0]
                for j in range(1, len(pollution_report.tables[adr_cnt].rows)):
                    row = pollution_report.tables[adr_cnt].rows[j]
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text)
                    row_df[row_text[0]] = row_text[1]
                df = df.append(row_df, ignore_index=True)
                adr_cnt += 1
    return df

In [57]:
df = pd.DataFrame(columns=columns)

for file in datafiles:
    df = df.append(addDate(file), ignore_index=True)

In [58]:
df

,Дата,Адрес,Оксид углерода,Оксид азота,Диоксид азота,Диоксид серы,Озон,Взвешенные частицы PM10,Взвешенные частицы PM2.5
0,18_07_2019,"Московский пр., дом 19",0.1,менее 0.1,0.6,менее 0.1,None,None,None
1,18_07_2019,"В.О.Средний пр., дом 74",0.1,менее 0.1,0.4,None,1.1,None,0.3
2,18_07_2019,"пр. КИМа, дом 26 лит. А",0.1,0.1,0.6,0.1,None,None,None
3,18_07_2019,"ул. Карбышева, дом 7",0.1,0.1,0.5,None,None,0.2,None
4,18_07_2019,"ул. Ольги Форш, дом 6",0.1,0.2,0.5,None,1.0,0.1,None
5,18_07_2019,"г. Колпино, Красная ул., дом 1а",0.1,0.1,0.3,0.1,None,0.6,None
6,18_07_2019,"пос. Металлострой, Железнодорожная ул., дом 13",0.1,0.1,0.5,менее 0.1,0.7,0.2,None
7,18_07_2019,"Малоохтинский пр., дом 98",менее 0.1,менее 0.1,0.2,менее 0.1,1.5,0.1,None
8,18_07_2019,"Индустриальный пр., д.64",0.1,0.1,0.4,менее 0.1,None,0.1,None
9,18_07_2019,"Уткин пр., д.16",0.1,менее 0.1,0.4,менее 0.1,None,0.2,None


In [5]:
#https://api.mapbox.com/geocoding/v5/mapbox.places/.json?bbox=30.0874,59.8003,30.556,60.07&access_token=
bbox = 'bbox=30.0874,59.8003,30.556,60.07'
api_auth_key = ''
base_url = 'https://api.mapbox.com'
geocoding_url = '/geocoding/v5/{endpoint}/{search_text}.json'